### Задание 1. Опишите модель данных, которая может лежать в основе данного приложения.  
Результатом выполнения задания стала следующая БД:

![database_1](database_1.png)

1)	Таблица **users** содержит информацию о имени, фамилии, отчестве пользователя, о номере телефона, электронной почте, а также о дополнительном телефоне. Все значения (кроме id естественно) представлены типом VARCHAR(50). NULL запрещён во всех полях, кроме alt_phone, т.к. он необязателен для заполнения в заявке. id является первичным ключом в этой и в других таблицах

2)	Таблица **users_passport** включает в себя все паспортные данные пользователя. Поле user_id связано с полем id в таблице users отношением 1 к 1 с каскадным удалением, т.к. один пользователь обычно не может иметь несколько паспортов, и при удалении пользователя из таблицы users нет смысла хранить него паспорт  
id – INT NOT NULL PRIMARY KEY   
user_id INT NOT NULL  
passport_series_number VARCHAR(50) NOT NULL – серия + номер паспорта. Решил не разделять по столбцам series и number для удобства. Лучше, чтобы при добавлении в БД строки автоматически конкатировались  
issue_date date NOT NULL – дата выдачи паспорта. Простая дата  
code VARCHAR(7) NOT NULL – код подразделения состоит из 6 символов и знака «-»  
issue_place VARCHAR(100) NOT NULL – кем выдан паспорт  
birthday date NOT NULL – дата рождения клиента  
birthplace VARCHAR(50) NOT NULL – место рождения  
registration VARCHAR(100) NOT NULL – адрес регистрации.  
Все значения NOT NULL. В заявлении необходимо указывать все данные  

3)	**users_job** хранит информацию о месте работы. Так же связана с users отношением 1 к 1 по полю users.id = users_job.user_id  
job_region (регион работы), company (место работы), post (должность) это VARCHAT(50) NOT NULL  
TIN (ИНН) – VARCHAR(10) NOT NULL, т.к. ИНН состоит ровно из 10 цифр.  
salary (зарплата) – INT NOT NULL – зарплату следует хранить как число. Возможно использовать float, но вряд - ли кто - то будет указывать зарплату с копейками  
date_of_employment (дата трудоустройства) – DATE NOT NULL  

4)	Таблица **applications** (заявки) связана с users отношением 1 ко многим через users.id = applications.user_id. Такой тип связи я выбрал потому, что один пользователь может иметь несколько кредитов.   
type (вид продукта) и purpose (цель кредита) это VARCHAR(50) NOT NULL. Можно использовать связь многие ко многим и вынести все возможные виды продуктов в отдельную таблицу (как я сделал это с дополнительными услугами), но мне кажется это нецелесообразным.   
sum (размер кредита) – тип INT NOT NULL  
term (срок кредита в месяцах) – SMALLINT NOT NULL. Выбрал SMALLINT для небольшой экономии памяти.  
total_sum (размер кредита с доп. услугами) INT NOT NULL  
interest_rat (ставка по кредиту) – FLOAT NOT NULL  
appl_date (дата составления заявки) date DEAFULT GETDATE(). Мне показалось, что необходимо учитывать дату, когда была заполнена заявка. Для этого я использовал функцию, возвращающую текущую дату.

5)	Таблица **options** содержит в себе список всех доступных дополнительных услуг с указанием их стоимости. Таблица состоит из  
id – стандартный первичный ключ  
name (название услуги) – VARCHAR(50) NOT NULL  
cost (стоимость услуги) FLOAT NOT NULL.  
Из – за того, что одна заявка может включать в себя несколько услуг, а также одна услуга может применяться к нескольким заявкам я использовал связь многие – ко – многим через дополнительную таблицу **application_options**.  
application_optons связана с таблицей application по ключу application_optons.application_id = application.id; а с таблицей options связь по ключу application_optons.option_id = options.id


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### Задание 2. Напишите SQL-запрос, который бы возвращал самый популярный вид продукта за текущий год.
Собственно, простой запрос:  
SELECT TOP(1) type, COUNT(*) as count FROM applications  
WHERE YEAR(appl_date) = 2022  
GROUP BY type  
ORDER BY count DESC  

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### Задание 3  
Предложите алгоритм для расчета графика аннуитетных платежей, напишите функцию, рассчитывающую график платежей с 5 столбцами: «Месяц», «Ежемесячный платеж», «Основной долг», «Долг по процентам» и «Остаток основного долга».  
Написал функцию с помощью Python и библиотеки Pandas  
Можно было использовать вывод переменных нарпямую из базы данных, но, как мне кажется, быстрее написать SQL запрос к самой БД, а оттуда выбрать интересующие нас 3 значения по id заявки: сумма кредита, ставка и срок. Протестировал функцию, указывая различные суммы, сроки, ставки - вроде всё работает. 

In [1]:
def credit_calc(credit_sum, rate, term): #Функция принимает 3 аргумента: сумму кредита, процентную ставку, срок кредита в месяцах
    import pandas as pd
    df = pd.DataFrame(columns=["Месяц", "Ежемесячный платёж", "Основной долг", "Долг по процентам", "Остаток основного долга"])
    balance = credit_sum #Задаём остаток
    m = 0 #Счётчик месяцев
    month_rate = rate/12/100 #Рассчитываем ежемесячную ставку
    month_payment = (month_rate*(1 + month_rate)**term)/((1 + month_rate)**term - 1)*balance #С помощью ежемесячной ставки считаем ежемесячный платёж на протяжении всего периода
    while balance >= month_payment: #balance показывает сколько осталось заплатить по кредиту. Как только остаток долга будет меньше ежемесячного платежа - цикл завершится
        m += 1
        lst = [m, month_payment,  month_payment-balance*month_rate, balance*month_rate, balance - (month_payment-balance*month_rate)] #Для добавления в датафрейм создаём список из 5 атрибутов
        df.loc[len(df)] = lst #Добавляем ранее созданный список в датафрейм
        balance = balance - (month_payment-balance*month_rate) #Рассчитываем новый остаток основного долга и начинаем новую итерацию
    lst = [term, month_payment, month_payment-balance*month_rate, balance*month_rate, 0] #Добавляем последний месяц в датафрейм
    df.loc[len(df)] = lst
    df = df.round(2).convert_dtypes().to_string(index=False) #Округляем значения и приводим датафрейм в строковый формат, т.е. примерно к тому результату, который требуется в задании
    if term % 10 in(2, 3, 4) and term not in(12, 13, 14, 112, 113, 114): #Склонения окончаний для слова "месяц" в зависимости от срока кредита
        print(" Сумма кредита:", credit_sum, "\n",  "Ставка:", rate, "%" "\n", "Срок:", term, "месяца", "\n", df)
    elif term % 10 == 1 and term not in(11, 111):
        print(" Сумма кредита:", credit_sum, "\n",  "Ставка:", rate, "%" "\n", "Срок:", term, "месяц", "\n", df)
    else:
        print(" Сумма кредита:", credit_sum, "\n",  "Ставка:", rate, "%" "\n", "Срок:", term, "месяцев", "\n", df)
credit_calc(100000, 13, 12) #Вызов функции

 Сумма кредита: 100000 
 Ставка: 13 %
 Срок: 12 месяцев 
  Месяц  Ежемесячный платёж  Основной долг  Долг по процентам  Остаток основного долга
     1             8931.73        7848.39            1083.33                 92151.61
     2             8931.73        7933.42             998.31                 84218.19
     3             8931.73        8019.36             912.36                 76198.82
     4             8931.73        8106.24             825.49                 68092.58
     5             8931.73        8194.06             737.67                 59898.53
     6             8931.73        8282.83              648.9                  51615.7
     7             8931.73        8372.56             559.17                 43243.14
     8             8931.73        8463.26             468.47                 34779.88
     9             8931.73        8554.95             376.78                 26224.94
    10             8931.73        8647.62              284.1                 17577